# Audiovisual Integration 

## Import libraries

In [ ]:
import pathlib
import matplotlib
from matplotlib import pyplot as plt
import glob 
import pandas as pd

# mne imports
import mne
from mne.datasets import sample
import mne.io

import numpy as np
matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from keras.callbacks import CSVLogger
from keras.models import load_model
from keras.losses import MeanSquaredError
#from keras.utils import to_categorical

# PyRiemann imports
# from pyriemann.estimation import XdawnCovariances
# from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix
#from sklearn.pipeline import make_pipeline
#from sklearn.linear_model import LogisticRegression


## Required installations

In [ ]:
!pip install tensorflow

In [ ]:
! pip install -U scikit-learn

In [ ]:
! pip install PyQt5

In [ ]:
!pip install pyriemann

In [ ]:
! pip install seaborn

## Load preprocessed EEGLAB data

In [ ]:
# Load file names of all subjects
folder_path = r'C:\Users\EliteBook HP840\Documents\MSc_Biomedical_Engineering\4rd Semester\Special Course\pipeline_data\data_preproc'
file_paths = glob.glob(folder_path +'/*.set') # List of all .set files in the folder that given by its folder_path

In [ ]:
# load all subjects
epochs = {}
for sub in range (len(file_paths)):
    epochs[sub] = mne.read_epochs_eeglab(file_paths[sub])

In [ ]:
# speech_mode = ["PP03", "PP09", "PP10", "PP11", "PP12", "PP13", "PP14", "PP15", "PP16", "PP17", "PP20", "PP25", "PP26", "PP28"]
speech_mode = [1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 23, 24, 26]
non_speech_mode = [0, 2, 3, 4, 5, 6, 16, 17, 19, 20, 21, 22, 25, 27]

## Identify the common channels


 SUBJECT INFO

 Speech mode: PP03, PP09, PP10, PP11, PP12, PP13, PP14, PP15, PP16, PP17, PP20, PP25, PP26, PP28

 Non-speech mode: PP02, PP04, PP05, PP06, PP07, PP08, PP18, PP19, PP21, PP22, PP23, PP24, PP27, PP29

In [ ]:
channels_nsp = {}; channels_sp = {};
ch_names_sp = []; ch_names_nsp = [];
for sub in non_speech_mode:
    channels_nsp[sub] = epochs[sub].ch_names
    ch_names_nsp.append(channels_nsp[sub])
    
for sub in speech_mode:
    channels_sp[sub] = epochs[sub].ch_names
    ch_names_sp.append(channels_sp[sub])


In [ ]:
from functools import reduce
nsp_channel_intersection = reduce(np.intersect1d, (ch_names_nsp))
print('Common channels between participants in non speech mode:', nsp_channel_intersection)

In [ ]:
sp_channel_intersection = reduce(np.intersect1d, (ch_names_sp))
print('Common channels between participants in speech mode:', sp_channel_intersection)

In [ ]:
channel_intersection= reduce(np.intersect1d, (nsp_channel_intersection, sp_channel_intersection))
print('Common channels across all subjects:', channel_intersection, '(', len(channel_intersection), ' channels)' )

## Retrieve data only from common channels

In [ ]:
for sub in range (len(file_paths)):
    epochs[sub].pick_channels(channel_intersection)

## Retrieve data for each condition and store in arrays

In [ ]:
# subtract the data for each stimuli and for each condition(and subject)

Tabi_A_SP = {}; Tabi_A_Tabi_V_SP = {}; Tabi_A_Tagi_V_SP = {}; Tabi_V_SP = {};
Tagi_A_SP = {}; Tagi_A_Tabi_V_SP = {}; Tagi_A_Tagi_V_SP = {}; Tagi_V_SP = {};

for sub in speech_mode:
    Tabi_A_SP[sub] = epochs[sub]['Tabi_A'].get_data()
    Tabi_A_Tabi_V_SP[sub] = epochs[sub]['Tabi_A_Tabi_V'].get_data()
    Tabi_A_Tagi_V_SP[sub] = epochs[sub]['Tabi_A_Tagi_V'].get_data()
    Tabi_V_SP[sub] = epochs[sub]['Tabi_V'].get_data()
    Tagi_A_SP[sub] = epochs[sub]['Tagi_A'].get_data()
    Tagi_A_Tabi_V_SP[sub] = epochs[sub]['Tagi_A_Tabi_V'].get_data()
    Tagi_A_Tagi_V_SP[sub] = epochs[sub]['Tagi_A_Tagi_V'].get_data()
    Tagi_V_SP[sub] = epochs[sub]['Tagi_V'].get_data()

Tabi_A_NSP = {}; Tabi_A_Tabi_V_NSP = {}; Tabi_A_Tagi_V_NSP = {}; Tabi_V_NSP = {};
Tagi_A_NSP = {}; Tagi_A_Tabi_V_NSP = {}; Tagi_A_Tagi_V_NSP = {}; Tagi_V_NSP = {};

for sub in non_speech_mode:
    Tabi_A_NSP[sub] = epochs[sub]['Tabi_A'].get_data()
    Tabi_A_Tabi_V_NSP[sub] = epochs[sub]['Tabi_A_Tabi_V'].get_data()
    Tabi_A_Tagi_V_NSP[sub] = epochs[sub]['Tabi_A_Tagi_V'].get_data()
    Tabi_V_NSP[sub] = epochs[sub]['Tabi_V'].get_data()
    Tagi_A_NSP[sub] = epochs[sub]['Tagi_A'].get_data()
    Tagi_A_Tabi_V_NSP[sub] = epochs[sub]['Tagi_A_Tabi_V'].get_data()
    Tagi_A_Tagi_V_NSP[sub] = epochs[sub]['Tagi_A_Tagi_V'].get_data()
    Tagi_V_NSP[sub] = epochs[sub]['Tagi_V'].get_data()


## Create trainset, testset and validation set

In [ ]:
# come back and exclede the non relevant incogruent condition!!!

SP_conditions = [Tabi_A_Tabi_V_SP,
                 Tabi_A_Tagi_V_SP,
                 Tagi_A_Tabi_V_SP,
                 Tagi_A_Tagi_V_SP]

NSP_conditions = [Tabi_A_Tabi_V_NSP,
                  Tabi_A_Tagi_V_NSP,
                  Tagi_A_Tabi_V_NSP,
                  Tagi_A_Tagi_V_NSP]   

### Subject-based split

In [ ]:
# subjects in each set
train_sp = [1, 7, 8, 9, 10, 11, 12, 13, 14, 15]
train_nsp = [0, 2, 3, 4, 5, 6, 16, 17, 19, 20]
test_sp = [18, 23]
test_nsp = [21, 22]
valid_sp = [24, 26]
valid_nsp = [25, 27]

In [ ]:
n=0; Xtr_sp = {}
for sub in train_sp:
    for cond in range(0,4):
        Xtr_sp[n] = SP_conditions[cond][sub]
        n=n+1

Xtrain_sp = Xtr_sp[0];     
for i in range(1, n):
    Xtrain_sp = np.append(Xtrain_sp, Xtr_sp[i], axis=0)
       
ytrain_sp = np.ones(len(Xtrain_sp))      

#######################################################################

n=0; Xtr_nsp = {}
for sub in train_nsp:
    for cond in range(0,4):
        Xtr_nsp[n] = NSP_conditions[cond][sub]
        n=n+1

Xtrain_nsp = Xtr_nsp[0];    
for i in range(1, n):
    Xtrain_nsp = np.append(Xtrain_nsp, Xtr_nsp[i], axis=0)

ytrain_nsp = np.zeros(len(Xtrain_nsp))

#######################################################################

n=0; Xtst_sp = {}
for sub in test_sp:
    for cond in range(0,4):
        Xtst_sp[n] = SP_conditions[cond][sub]
        n=n+1

Xtest_sp = Xtst_sp[0]
for i in range(1, n):
    Xtest_sp = np.append(Xtest_sp, Xtst_sp[i], axis=0)

ytest_sp = np.ones(len(Xtest_sp))

#######################################################################

n=0; Xtst_nsp = {}
for sub in test_nsp:
    for cond in range(0,4):
        Xtst_nsp[n] = NSP_conditions[cond][sub]
        n=n+1

Xtest_nsp = Xtst_nsp[0]
for i in range(1, n):
    Xtest_nsp = np.append(Xtest_nsp, Xtst_nsp[i], axis=0)
                   
ytest_nsp =np.zeros(len(Xtest_nsp))

#######################################################################
        
n=0; Xval_sp = {}
for sub in valid_sp:
    for cond in range(0,4):
        Xval_sp[n] = SP_conditions[cond][sub]
        n=n+1

Xvalid_sp = Xval_sp[0]
for i in range(1, n):
    Xvalid_sp = np.append(Xvalid_sp, Xval_sp[i], axis=0)

yvalid_sp = np.ones(len(Xvalid_sp))

#######################################################################

n=0; Xval_nsp = {}
for sub in valid_nsp:
    for cond in range(0,4):
        Xval_nsp[n] = NSP_conditions[cond][sub]
        n=n+1

Xvalid_nsp = Xval_nsp[0]
for i in range(1, n):
    Xvalid_nsp = np.append(Xvalid_nsp, Xval_nsp[i], axis=0)

yvalid_nsp = np.zeros(len(Xvalid_nsp))

### Trial-based split

In [ ]:
n=0; Xtr_sp={}; Xtst_sp={}; Xval_sp={};
for sub in speech_mode:  
    for cond in range(0,4):        
        Xsp = SP_conditions[cond][sub]
        Xtr_sp[n] = Xsp[0:round(0.8*len(Xsp)), 0:, 0:]
        Xtst_sp[n] = Xsp[round(0.8*len(Xsp)):round(0.9*len(Xsp)), 0:, 0:]
        Xval_sp[n] = Xsp[round(0.9*len(Xsp)):, 0:, 0:]
        n=n+1
        
Xtrain_sp = Xtr_sp[0]; Xtest_sp = Xtst_sp[0]; Xvalid_sp = Xval_sp[0];      
for i in range(1, n):
    Xtrain_sp = np.append(Xtrain_sp, Xtr_sp[i], axis=0)
    Xtest_sp = np.append(Xtest_sp, Xtst_sp[i], axis=0)
    Xvalid_sp = np.append(Xvalid_sp, Xval_sp[i], axis=0)
       
print(Xtrain_sp.shape); print(Xtest_sp.shape); print(Xvalid_sp.shape) 

ytrain_sp = np.ones(len(Xtrain_sp))
ytest_sp = np.ones(len(Xtest_sp))
yvalid_sp = np.ones(len(Xvalid_sp))

n=0; Xtr_nsp={}; Xtst_nsp={}; Xval_nsp={};
for sub in non_speech_mode:  
    for cond in range(0,4):        
        Xnsp = NSP_conditions[cond][sub]
        Xtr_nsp[n] = Xnsp[0:round(0.8*len(Xnsp)), 0:, 0:]
        Xtst_nsp[n] = Xnsp[round(0.8*len(Xnsp)):round(0.9*len(Xnsp)), 0:, 0:]
        Xval_nsp[n] = Xnsp[round(0.9*len(Xnsp)):, 0:, 0:]
        n=n+1
        
Xtrain_nsp =  Xtr_nsp[0]; Xtest_nsp=Xtst_nsp[0]; Xvalid_nsp = Xval_nsp[0];             
for i in range(1, n):
    Xtrain_nsp = np.append(Xtrain_nsp, Xtr_nsp[i], axis=0)
    Xtest_nsp = np.append(Xtest_nsp, Xtst_nsp[i], axis=0)
    Xvalid_nsp = np.append(Xvalid_nsp, Xval_nsp[i], axis=0)

     
print(Xtrain_nsp.shape); print(Xtest_nsp.shape); print(Xvalid_nsp.shape) 

ytrain_nsp = np.zeros(len(Xtrain_nsp))
ytest_nsp = np.zeros(len(Xtest_nsp))
yvalid_nsp = np.zeros(len(Xvalid_nsp))

### Trial-based split No.2

In [ ]:
n=0; Xtr_sp={}; Xtst_sp={}; Xval_sp={};
for sub in speech_mode:  
    for cond in range(0,4):        
        Xsp = SP_conditions[cond][sub]
        Xtr_sp[n] = Xsp[0:round(0.5*len(Xsp)), 0:, 0:]
        Xtst_sp[n] = Xsp[round(0.5*len(Xsp)):round(0.75*len(Xsp)), 0:, 0:]
        Xval_sp[n] = Xsp[round(0.75*len(Xsp)):, 0:, 0:]
        n=n+1
        
Xtrain_sp = Xtr_sp[0]; Xtest_sp = Xtst_sp[0]; Xvalid_sp = Xval_sp[0];      
for i in range(1, n):
    Xtrain_sp = np.append(Xtrain_sp, Xtr_sp[i], axis=0)
    Xtest_sp = np.append(Xtest_sp, Xtst_sp[i], axis=0)
    Xvalid_sp = np.append(Xvalid_sp, Xval_sp[i], axis=0)
       
print(Xtrain_sp.shape); print(Xtest_sp.shape); print(Xvalid_sp.shape) 

ytrain_sp = np.ones(len(Xtrain_sp))
ytest_sp = np.ones(len(Xtest_sp))
yvalid_sp = np.ones(len(Xvalid_sp))

n=0; Xtr_nsp={}; Xtst_nsp={}; Xval_nsp={};
for sub in non_speech_mode:  
    for cond in range(0,4):        
        Xnsp = NSP_conditions[cond][sub]
        Xtr_nsp[n] = Xnsp[0:round(0.5*len(Xnsp)), 0:, 0:]
        Xtst_nsp[n] = Xnsp[round(0.5*len(Xnsp)):round(0.75*len(Xnsp)), 0:, 0:]
        Xval_nsp[n] = Xnsp[round(0.75*len(Xnsp)):, 0:, 0:]
        n=n+1
        
Xtrain_nsp =  Xtr_nsp[0]; Xtest_nsp=Xtst_nsp[0]; Xvalid_nsp = Xval_nsp[0];             
for i in range(1, n):
    Xtrain_nsp = np.append(Xtrain_nsp, Xtr_nsp[i], axis=0)
    Xtest_nsp = np.append(Xtest_nsp, Xtst_nsp[i], axis=0)
    Xvalid_nsp = np.append(Xvalid_nsp, Xval_nsp[i], axis=0)

     
print(Xtrain_nsp.shape); print(Xtest_nsp.shape); print(Xvalid_nsp.shape) 

ytrain_nsp = np.zeros(len(Xtrain_nsp))
ytest_nsp = np.zeros(len(Xtest_nsp))
yvalid_nsp = np.zeros(len(Xvalid_nsp))

### Final data onfiguration

In [ ]:
X_train = np.append(Xtrain_sp[:,:,127:-1] , Xtrain_nsp[:,:,127:-1], axis=0 )
y_train1= np.append(ytrain_sp, ytrain_nsp, axis=0 )

    
X_test = np.append(Xtest_sp[:,:,127:-1] , Xtest_nsp[:,:,127:-1], axis=0 )
y_test1= np.append(ytest_sp, ytest_nsp, axis=0 )
 
X_validate = np.append(Xvalid_sp[:,:,127:-1] , Xvalid_nsp[:,:,127:-1], axis=0 )
y_validate1= np.append(yvalid_sp, yvalid_nsp, axis=0)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_validate.shape)
print(y_train1)
print(y_test1)
print(y_validate1)

In [ ]:
#one hot encoding
y_train2= np.append(ytrain_nsp, ytrain_sp , axis=0 )
y_test2= np.append(ytest_nsp, ytest_sp , axis=0 )
y_validate2= np.append(yvalid_nsp, yvalid_sp, axis=0)

In [ ]:
print(y_train2)
print(y_test2)
print(y_validate2)

In [ ]:
Y_train= np.stack((y_train1, y_train2) , axis=-1)
Y_test= np.stack((y_test1, y_test2) , axis=-1 )
Y_validate= np.stack((y_validate1, y_validate2), axis=-1)

In [ ]:
print(Y_train.shape)
print(Y_test.shape)
print(Y_validate)

## EEGNet Models

In [ ]:
"""
 ARL_EEGModels - A collection of Convolutional Neural Network models for EEG
 Signal Processing and Classification, using Keras and Tensorflow

 Requirements:
    (1) tensorflow == 2.X (as of this writing, 2.0 - 2.3 have been verified
        as working)
 
 To run the EEG/MEG ERP classification sample script, you will also need

    (4) mne >= 0.17.1
    (5) PyRiemann >= 0.2.5
    (6) scikit-learn >= 0.20.1
    (7) matplotlib >= 2.2.3
    
 To use:
    
    (1) Place this file in the PYTHONPATH variable in your IDE (i.e.: Spyder)
    (2) Import the model as
        
        from EEGModels import EEGNet    
        
        model = EEGNet(nb_classes = ..., Chans = ..., Samples = ...)
        
    (3) Then compile and fit the model
    
        model.compile(loss = ..., optimizer = ..., metrics = ...)
        fitted    = model.fit(...)
        predicted = model.predict(...)

 Portions of this project are works of the United States Government and are not
 subject to domestic copyright protection under 17 USC Sec. 105.  Those 
 portions are released world-wide under the terms of the Creative Commons Zero 
 1.0 (CC0) license.  
 
 Other portions of this project are subject to domestic copyright protection 
 under 17 USC Sec. 105.  Those portions are licensed under the Apache 2.0 
 license.  The complete text of the license governing this material is in 
 the file labeled LICENSE.TXT that is a part of this project's official 
 distribution. 
"""


def EEGNet(nb_classes, Chans = 36, Samples = 128, 
             dropoutRate = 0.25, kernLength = 32, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta

    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:
        
        1. Depthwise Convolutions to learn spatial filters within a 
        temporal convolution. The use of the depth_multiplier option maps 
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn 
        spatial filters within each filter in a filter-bank. This also limits 
        the number of free parameters to fit when compared to a fully-connected
        convolution. 
        
        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions. 
        
    
    While the original paper used Dropout, we found that SpatialDropout2D 
    sometimes produced slightly better results for classification of ERP 
    signals. However, SpatialDropout2D significantly reduced performance 
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.
        
    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the 
    kernel lengths for double the sampling rate, etc). Note that we haven't 
    tested the model performance with this rule so this may not work well. 
    
    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.

    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D. 

    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.

    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

## Fitting the model

In [ ]:
"""
 Sample script using EEGNet to classify Event-Related Potential (ERP) EEG data
 from a four-class classification task, using the sample dataset provided in
 the MNE [1, 2] package:
     https://martinos.org/mne/stable/manual/sample_dataset.html#ch-sample-data
   
 The four classes used from this dataset are:
     LA: Left-ear auditory stimulation
     RA: Right-ear auditory stimulation
     LV: Left visual field stimulation
     RV: Right visual field stimulation

 The code to process, filter and epoch the data are originally from Alexandre
 Barachant's PyRiemann [3] package, released under the BSD 3-clause. A copy of 
 the BSD 3-clause license has been provided together with this software to 
 comply with software licensing requirements. 
 
 When you first run this script, MNE will download the dataset and prompt you
 to confirm the download location (defaults to ~/mne_data). Follow the prompts
 to continue. The dataset size is approx. 1.5GB download. 
 
 For comparative purposes you can also compare EEGNet performance to using 
 Riemannian geometric approaches with xDAWN spatial filtering [4-8] using 
 PyRiemann (code provided below).

 [1] A. Gramfort, M. Luessi, E. Larson, D. Engemann, D. Strohmeier, C. Brodbeck,
     L. Parkkonen, M. Hämäläinen, MNE software for processing MEG and EEG data, 
     NeuroImage, Volume 86, 1 February 2014, Pages 446-460, ISSN 1053-8119.

 [2] A. Gramfort, M. Luessi, E. Larson, D. Engemann, D. Strohmeier, C. Brodbeck, 
     R. Goj, M. Jas, T. Brooks, L. Parkkonen, M. Hämäläinen, MEG and EEG data 
     analysis with MNE-Python, Frontiers in Neuroscience, Volume 7, 2013.

 [3] https://github.com/alexandrebarachant/pyRiemann. 

 [4] A. Barachant, M. Congedo ,"A Plug&Play P300 BCI Using Information Geometry"
     arXiv:1409.0107. link

 [5] M. Congedo, A. Barachant, A. Andreev ,"A New generation of Brain-Computer 
     Interface Based on Riemannian Geometry", arXiv: 1310.8115.

 [6] A. Barachant and S. Bonnet, "Channel selection procedure using riemannian 
     distance for BCI applications," in 2011 5th International IEEE/EMBS 
     Conference on Neural Engineering (NER), 2011, 348-351.

 [7] A. Barachant, S. Bonnet, M. Congedo and C. Jutten, “Multiclass 
     Brain-Computer Interface Classification by Riemannian Geometry,” in IEEE 
     Transactions on Biomedical Engineering, vol. 59, no. 4, p. 920-928, 2012.

 [8] A. Barachant, S. Bonnet, M. Congedo and C. Jutten, “Classification of 
     covariance matrices using a Riemannian-based kernel for BCI applications“, 
     in NeuroComputing, vol. 112, p. 172-178, 2013.


 Portions of this project are works of the United States Government and are not
 subject to domestic copyright protection under 17 USC Sec. 105.  Those 
 portions are released world-wide under the terms of the Creative Commons Zero 
 1.0 (CC0) license.  
 
 Other portions of this project are subject to domestic copyright protection 
 under 17 USC Sec. 105.  Those portions are licensed under the Apache 2.0 
 license.  The complete text of the license governing this material is in 
 the file labeled LICENSE.TXT that is a part of this project's official 
 distribution. 
"""
from mne import io

# while the default tensorflow ordering is 'channels_last' we set it here
# to be explicit in case if the user has changed the default ordering
K.set_image_data_format('channels_last')

##################### Process, filter and epoch the data ######################
#data_path = sample.data_path()

# Set parameters and read data
# raw_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
# event_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'
# tmin, tmax = -0., 1
# event_id = dict(aud_l=1, aud_r=2, vis_l=3, vis_r=4)

# Setup for reading the raw data
# raw = io.Raw(raw_fname, preload=True, verbose=False)
# raw.filter(2, None, method='iir')  # replace baselining with high-pass
# events = mne.read_events(event_fname)

# raw.info['bads'] = ['MEG 2443']  # set bad channels
# picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                        exclude='bads')

# # Read epochs
# epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
#                     picks=picks, baseline=None, preload=True, verbose=False)
# labels = epochs.events[:, -1]

# # extract raw data. scale by 1000 due to scaling sensitivity in deep learning
# X = epochs.get_data()*1000 # format is in (trials, channels, samples)
# y = labels

kernels, chans, samples = 1, 36, 128
#we are using half the samples, to exclude the first ones as noise

# take 50/25/25 percent of the data to train/validate/test
# X_train      = X[0:144,]
# Y_train      = y[0:144]
# X_validate   = X[144:216,]
# Y_validate   = y[144:216]
# X_test       = X[216:,]
# Y_test       = y[216:]




############################# EEGNet portion ##################################

# convert labels to one-hot encodings.
# Y_train      = np_utils.to_categorical(Y_train-1)
# Y_validate   = np_utils.to_categorical(Y_validate-1)
# Y_test       = np_utils.to_categorical(Y_test-1)

# convert data to NHWC (trials, channels, samples, kernels) format. Data 
# contains 60 channels and 151 time-points. Set the number of kernels to 1.
# N: number of images in the batch
# H: height of the image
# W: width of the image
# C: number of channels of the image (ex: 3 for RGB, 1 for grayscale...)
# X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
# X_validate   = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
# X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
   
# print('X_train shape:', X_train.shape)
# print(X_train.shape[0], 'train samples')
# print(X_test.shape[0], 'test samples')

# configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = 0.25, kernLength = 32, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

# compile the model and set the optimizers
#model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer='adam', 
              #metrics = ['accuracy'])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])
# count number of parameters in the model
numParams    = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

###############################################################################
# if the classification task was imbalanced (significantly more trials in one
# class versus the others) you can assign a weight to each class during 
# optimization to balance it out. This data is approximately balanced so we 
# don't need to do this, but is shown here for illustration/completeness. 
###############################################################################

# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1
class_weights = {0:1, 1:1}

################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
# Riemannian geometry classification (below)
################################################################################


#csv_logger = CSVLogger('training.log', separator=',', append=False)



fittedModel = model.fit(X_train, Y_train, batch_size = 100, epochs = 500, 
                        verbose = 2, validation_data=(X_validate, Y_validate),
                        callbacks=[checkpointer], class_weight = class_weights)

# load optimal weights
model.load_weights('/tmp/checkpoint.h5')

###############################################################################
# can alternatively used the weights provided in the repo. If so it should get
# you 93% accuracy. Change the WEIGHTS_PATH variable to wherever it is on your
# system.
###############################################################################

#WEIGHTS_PATH = "/path/to/EEGNet-8-2-weights.h5 "
#model.load_weights(WEIGHTS_PATH)


###############################################################################
# evaluate the model on the test set.
###############################################################################

# Evaluate the model on the test data using `evaluate`

# print("Evaluate on test data")
# results=model.evaluate(X_test,Y_test,batch_size=20)
# print("test loss, test acc:",results)

###############################################################################
# make prediction on test set.
###############################################################################

probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == Y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))


############################# PyRiemann Portion ##############################

# # code is taken from PyRiemann's ERP sample script, which is decoding in 
# # the tangent space with a logistic regression

# n_components = 2  # pick some components

# # set up sklearn pipeline
# clf = make_pipeline(XdawnCovariances(n_components),
#                     TangentSpace(metric='riemann'),
#                     LogisticRegression())

# preds_rg     = np.zeros(len(Y_test))

# # reshape back to (trials, channels, samples)
# X_train      = X_train.reshape(X_train.shape[0], chans, samples)
# X_test       = X_test.reshape(X_test.shape[0], chans, samples)

# # train a classifier with xDAWN spatial filtering + Riemannian Geometry (RG)
# # labels need to be back in single-column format
# clf.fit(X_train, Y_train.argmax(axis = -1))
# preds_rg     = clf.predict(X_test)

# # Printing the results
# acc2         = np.mean(preds_rg == Y_test.argmax(axis = -1))
# print("Classification accuracy: %f " % (acc2))

# # plot the confusion matrices for both classifiers
# names        = ['audio left', 'audio right', 'vis left', 'vis right']
# plt.figure(0)
# plot_confusion_matrix(preds, Y_test.argmax(axis = -1), names, title = 'EEGNet-8,2')

# plt.figure(1)
# plot_confusion_matrix(preds_rg, Y_test.argmax(axis = -1), names, title = 'xDAWN + RG')


## Plotting

In [ ]:
import pandas as pd

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(fittedModel.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
folder_path = r'C:\Users\EliteBook HP840\Documents\MSc_Biomedical_Engineering\4rd Semester\Deep Learning\Project\history.csv'
df_hist = pd.read_csv(folder_path)

In [ ]:
# plot accuracy during training
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.plot(df_hist['accuracy'], label='train')
plt.plot(df_hist['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
# summarize history for loss
plt.plot(df_hist['loss'])
plt.plot(df_hist['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# plot_confusion_matrix(preds, Y_test.argmax(axis = -1), names, title = 'EEGNet-8,2')